In [3]:
from IPython.display import SVG
from types import SimpleNamespace
import numpy as np
import holoviews as hv
from holoviews import opts
# import kwant
import numpy as np
# import tinyarray as tiny
from types import SimpleNamespace

# Goal
#### Calculate transport properties of skyrmion with Kwant

## To do
- Try to find a signature of finite size
- Large disk -> vary V_z
- Plot DoS of kwant system vs. analytic
- Verify that kwant system with symmetry gives exactly the same as analytic (for the leads)
- Implement complete skyrmion magnetic texture and plot results

*NOTE: A dynamic version of the plots below is coded, but a live python server is needed to run it. Therefore the sliders below only take on a couple of values instead of being continuous (this journal would be way to memory expensive if I keep importing all the images)*

In [8]:
hv.extension('bokeh', logo=False)
dims = dict(kx = 'k_x',ky = 'k_y', mu = 'µ', delta = 'Δ', t = 't', E='ε',mu_t = 'µ/t')
def dispersion_1d(L=30,t=1,ky=0,mu=0,delta=0, j=0):
    kx = np.linspace(-np.pi,np.pi,L)
    e_up = j + np.sqrt( (2*t*(np.cos(kx)+np.cos(ky)-2)-mu)**2 + delta**2 )
    e_down = -j + np.sqrt( (2*t*(np.cos(kx)+np.cos(ky)-2)-mu)**2 + delta**2 )
    h_up = -e_down #holes with up spin
    h_down = -e_up #holes with down spin
    
    plot_opts = dict(width=350, height=350, xticks=[(-np.pi, '-π'),(0, '0'),(np.pi, 'π')], default_tools=['wheel_zoom','reset', 'save'], framewise=True)
    e_bands = hv.Curve((kx,e_up), kdims=[dims['kx'],dims['E']]).opts(**plot_opts,color='deepskyblue')*\
                hv.Curve((kx,e_down), kdims=[dims['kx'],dims['E']]).opts(**plot_opts,color='deepskyblue', line_dash='dashed')
    h_bands = hv.Curve((kx,h_up), kdims=[dims['kx'],dims['E']]).opts(**plot_opts,color='coral')*\
            hv.Curve((kx,h_down), kdims=[dims['kx'],dims['E']]).opts(**plot_opts,color='coral', line_dash='dashed')
    
    bands_plot = e_bands*h_bands
    fermi_level = hv.HLine(mu).opts(**plot_opts, color='black', line_dash='dashed', line_width=1.5)
    
    return (bands_plot*fermi_level).opts(title='Spectrum 1D L={:d}'.format(L),xlim=(-np.pi,np.pi))

def dos_bulk_2d(L=30,t=1,mu=0,delta=0,j=0, bandwidth=0.1, n_bins=None):
    x,y = np.meshgrid(np.linspace(-np.pi,np.pi,L),np.linspace(-np.pi,np.pi,L))
    e_up = j + np.sqrt( (2*t*(np.cos(x)+np.cos(y)-2)-mu)**2 + delta**2 )
    e_down = -j + np.sqrt( (2*t*(np.cos(x)+np.cos(y)-2)-mu)**2 + delta**2 )
    spec = np.asarray([e_up,-e_up, e_down, -e_down]).flatten()
    
    plot_opts=dict(width=350, height=350,invert_axes=True, default_tools=['wheel_zoom','reset', 'save'], framewise=True)
    fermi_level = hv.VLine(mu).opts(**plot_opts, color='black', line_dash='dashed', line_width=1.5)
    if n_bins == None:
#         plot_opts['bandwidth']= bandwidth
        distr = hv.Distribution(spec, kdims=dims['E']).opts(bandwidth=bandwidth).opts(plot=plot_opts)
        plot = (distr*fermi_level)
    else:
        hist = hv.Histogram(np.histogram(spec, bins=n_bins, density=True), kdims=dims['E'], vdims='Density').opts(plot=plot_opts)
        plot = (hist*fermi_level)
        
    return plot.opts(title='DoS 2D ({}x{})'.format(L,L))

dispersion_live = hv.DynamicMap(dispersion_1d, kdims=['t','delta', 'mu', 'j','ky']).redim.range(t=(1,4), mu=(-3,3), delta=(0,4), j=(0,3.), ky=(0,3.14))
dos_live = hv.DynamicMap(dos_bulk_2d, kdims=['t', 'mu','delta', 'j']).redim.range(t=(1,4), mu=(-3,3), delta=(0,4), j=(0,3.))
# live_plot = (dispersion_live+dos_live).opts(title='',merge_tools=True)
# live_plot

# Uncomment below for Static version (useful for NBviewer for example)
dispersion_static = hv.HoloMap(dispersion_live[{1,3},{0,3},{0,3},{0,1,2,3},{0,3.14}])
dos_static = hv.HoloMap(dos_live[{1,3},{0,3},{0,3},{0,1,2,3}])
static_plot = (dispersion_static + dos_static).opts(title='', merge_tools=True)
static_plot


:Layout
   .HoloMap.I  :HoloMap   [t,delta,mu,j,ky]
      :Overlay
         .Curve.I   :Curve   [k_x,ε]   (y)
         .Curve.II  :Curve   [k_x,ε]   (y)
         .Curve.III :Curve   [k_x,ε]   (y)
         .Curve.IV  :Curve   [k_x,ε]   (y)
         .HLine.I   :HLine   [x,y]
   .HoloMap.II :HoloMap   [t,mu,delta,j]
      :Overlay
         .Distribution.I :Distribution   [ε]   (Density)
         .VLine.I        :VLine   [x,y]

## 24 May
- Let's try to find at which size we can definitely see some finite-size effect in Kwant

## 23 May
- Found out how to make live plots (computationally a bit more expensive, but at least it doesnt keep all the images in the cache) and how to take some of them and convert them to static (but still interactive) plots.
- Introduced the magnetic field term in the z-direction and diagonalised the resulting matrix using mathematica (I was on the way doing it by hand, but messed up one definition which made it quite hard to continue. Also squaring this matrix didn't simplify it as was the case for a Hamiltonian with just factors acting on PH-space). The eigenvectors are naturally just linear combinations of two operators due to the pairing term. 
- Introduced the magnetic coupling in the bulk plots, see above

## 22 May
- Made some nice interactive plots to show the dispersion and the density of states of the bulk of the disk
- According to those plots, the spectrum I obtain from kwant is right in the sense that the bulk energies correspond to what we are supposed to find analytically.

## 21 May
- Fixed everything up to generate the spectra on 20 May.
- Trying to get continuous spectra to see if it corroborates the spectra of 20 May
- The eigenvectors that result from the spectra obtained on the 20th don't make any sense
- Started setting up the sparse solver: Solving disk with radius 100 for the 30 highest eigenvalues & vectors takes 150 seconds (though no difficult Hamiltonians yet)

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}$$
## 20 May
- Verified that the kwant system up to now generates the right Hamiltonian without the skyrmion, including only a magnetic field in the z-direction: <br> $$H=\sum_{i,j}-t\tau_z\ket{n.n.}\bra{n.n.} + \{(4t-\mu)\tau_z + J\sigma_z\ +\Delta \tau_x\}\ket{i,j}\bra{i,j} $$
<tab>The generated spectra are plotted below (something strange is happening for the variation of mu, I think there must be a minus sign wrong somewhere):<br>
![Spec_delta](Jupyter/fig/spectrum_delta.svg)![Spec_j](Jupyter/fig/spectrum_j.svg)![Spec_mu](Jupyter/fig/spectrum_mu.svg)![Spec_t](Jupyter/fig/spectrum_t.svg)

## 17 May
- Got the Hamiltonian set up in kwant to also take the interaction with the skyrmion


## 16 May
- Learned how to discretize Hamiltonians
- Simulated transmission in a Rashba wire, see <a href='https://nbviewer.jupyter.org/github/Arfor/m1internship/blob/master/Jupyter/Magnetic_texture.ipynb'>Rashba_wire.ipynb</a>
- First spectrum obtained from skyrmion on a square lattice with circular geometry: <br>
    A radius of r=15 gives 697 sites on this lattice, so the Hamiltonian in nambu basis has a size (2788,2788) [check number of site with *len(sys.onsites)*] <br>
    timing: solving this Hamiltonian with numpy.linalg takes 7,5 seconds
- I need to rewrite it in position basis, then the Hamiltonian should be (2n,2n) instead of (4n,4n).

## 15 May
- Read Aguado review
- Fixed skyrmion definition of 14 May

## 14 May
- Finished the magnetic texture of the skyrmion, although plotting it with plt.quiver which plots the vector field with arrows was very annoying, so to check, I just made it such that you can check the x,y,z components of the magnetic field:

In [13]:
hv.extension('matplotlib', logo=False)
params = SimpleNamespace(t=1, mu=-0.1, j=0.02, delta=0.2, B_0=0.1, aziwinding=1, radiwinding=1,radius=20)
def magn_skyrmion(position,params):
    x,y = position
    t = np.arctan2(x,y)
    q = params.aziwinding
    p = params.radiwinding
    R = params.radius
    r = np.sqrt(x**2 + y**2)
    B = [np.sin(np.pi*p*(r/R))*np.cos(q*t), np.sin(np.pi*p*(r/R))*np.sin(q*t), np.cos(np.pi*p*(r/R))]
    return B

def texture(params, q=1,p=1):
    init = []
    params.aziwinding=q
    params.radiwinding=p
    r= params.radius
    x = np.arange(-r,r,1)
    y = np.arange(-r,r,1)
    for i in x:
        for j in y:
            init.append(magn_skyrmion((i,j),params))
    return np.reshape(np.asarray(init),(len(x),len(y),3)) #reshape Fortran style, otherwise x & y are swapped

r= params.radius
plot_opts = dict(width=300, height=300)
style_opts= dict(colorbar=True, cmap='inferno')
hv.HoloMap({(i,q,p): hv.Image(texture(params,q=q,p=p)[:,:,i], bounds=(r,r,-r,-r)).opts(style=style_opts, plot = plot_opts) for i in range(3) \
            for q in range(1,4) for p in range(1,4)}, kdims=['Component','Azimuthal Winding Number', 'Radial Winding Number'])

:HoloMap   [Component,Azimuthal Winding Number,Radial Winding Number]
   :Image   [x,y]   (z)

## 13 May
- kwant.Builder doesn't seem to like it yet when you want to pass a dictionary to the params parameter, so you have to use args[p], where p can be a dictionary/simplenamespace. There is an error when trying to define the hopping terms on the disk -> dimension of onsite terms doesn't match those of the hopping ones.
- A bit of confusion regarding the Hamiltonian as defined in the skyrmion paper: 
\begin{equation}
H(\mathbf{r}) =   \left(  - \frac{1}{2m}\nabla^2 - \mu \right)\tau_z + J \sigma_{\mathbf{r}} \cdot \mathbf{n(r)} + \Delta_0 \tau_x.
\end{equation}
Using the Nambu basis
\begin{equation}
\psi^{\dagger}(\mathbf{r}) = \left( c^{\dagger}_{\uparrow}(\mathbf{r}) c^{\dagger}_{\downarrow}(\mathbf{r}) \ 
    c_{\downarrow}(\mathbf{r}) -c_{\uparrow}(\mathbf{r}) \right), 
\end{equation}
how are the (4x4 matrices) $\tau_z$, $\tau_x$ and $\sigma_x,\sigma_y,\sigma_z$ defined? 

*The $\sigma$'s and $\tau$'s act on the spin space and particle-hole space only. This means that in the basis as defined above, the $\sigma$'s are just on the right of the Kronecker product with the identity whereas the tau's are one the left of the Kronecker product with the identity, i.e. $${}^{(4)}\sigma_\alpha = I_2 \otimes {}^{(2)}\sigma_\alpha \\ {}^{(4)}\tau_\alpha = {}^{(2)}\sigma_\alpha \otimes I_2.$$*

## 10 May
 - Finished the Kwant whitepaper
 - Playing around with Kwant on a <a href='https://nbviewer.jupyter.org/github/Arfor/m1internship/blob/master/Jupyter/Kwant_disk.ipynb'> disk</a> -> Made basic setup for magnetic texture on disk and defined function to fill onsite terms of Hamiltonian
 ![Disks](Jupyter/fig/disks_r=10.svg)


## 9 May
 - Made sure the jupyter notebooks are available to view online by making a repo on GitHub
 - Finished 1st read of Skyrmion paper
 - Started reading the <a href='https://arxiv.org/abs/1309.2926'> whitepaper of kwant </a> (2014)
 - Got circular geometry on a square lattice structure using kwant.Builder


## 7 May
- Started reading *Topological Superconductivity with Deformable Magnetic Skyrmions - Garnier, Mesaros, Simon*

The reason why such a complicated magnetic texture is being researched is because it is a way to achieve p-wave coupling in a superconductor, which is reducible to Kitaev's model. Only in p-wave superconductors we find topological invariants and Majorana edge modes, but we have no evidence as of now that they exist in nature. Therefore, p-waves must be engineered and using the magnetic texture of a skyrmion, which has already been made in the lab, is one possible way of achieving this. 
The crux to achieving p-wave superconductivity is to deform the Hamiltonian in such a way that the only pairing that can happen at a certain energy is between two spin-independent electrons:
$$
\Delta_s c^{\dagger}_{k\uparrow} c^{\dagger}_{-k,\downarrow} \to \Delta_p c^{\dagger}_{k}c^{\dagger}_{-k}
$$
However, just splitting the spin values with for example a Zeeman term in the z-direction is not enough. For example, a simple Zeeman term in the z-direction would split the electrons into up- and down-spin, but since we proximitize the skyrmion with an s-wave superconductor, the electrons cannot be coupled all of sudden with the same spin (p-wave). <br>
*Not too sure about this: If two Kitaev chains with Majorana edges would be close to each other, they would be able to talk and destroy the zero-energy modes. For a simple Zeeman term there would be two copies of the Kitaev chain for a certain energy $\epsilon$ in close proximity which would destroy the zero-mode.*
$$H_B = -B_z \sigma_z c^{\dagger}_{k\sigma}c^{}_{k\sigma}$$

In [17]:
hv.extension('matplotlib', logo=False)
k = np.linspace(-np.pi,np.pi,200)
e_up = k**2 - 1
e_down = k**2 + 1
left,right = (k-1)**2, (k+1)**2

plot1 = hv.Overlay([hv.Path((k,e_up), kdims=['$k_z$','$\epsilon$']),hv.Path((k,e_down)),hv.HLine(0).opts(linestyle='--',color='grey'),hv.VLine(0).opts(linestyle='--', color='grey')])


plot2 = hv.Overlay([hv.Curve((k,left), kdims=['$k_x$','$\epsilon$']),hv.Curve((k,right)),hv.HLine(0).opts(linestyle='--',color='grey'),hv.VLine(0).opts(linestyle='--', color='grey')])
plot1.opts(yticks=[(0,0)],xticks=[(-np.pi, '$-\pi$'),(0, 0), (np.pi, '$\pi$')],ylim=(-2,6)) + plot2.opts(yticks=[(0,0)],xticks=[(-np.pi, '$-\pi$'),(0, 0), (np.pi, '$\pi$')], ylim=(0,6))

:Layout
   .Overlay.I  :Overlay
      .Path.I  :Path   [$k_z$,$\epsilon$]
      .Path.II :Path   [x,y]
      .HLine.I :HLine   [x,y]
      .VLine.I :VLine   [x,y]
   .Overlay.II :Overlay
      .Curve.I  :Curve   [$k_x$,$\epsilon$]   (y)
      .Curve.II :Curve   [x]   (y)
      .HLine.I  :HLine   [x,y]
      .VLine.I  :VLine   [x,y]

**_Question_**: *If we go to $\epsilon=0$ now, wouldn't you get p-wave coupling?* <br>
**_Answer_**: *No, proximitizing with an s-wave superconductor makes it impossible to immediately get coupling of electrons with equal spins*

So we have to add some spices in order to be able to get nearer to the Kitaev model --> magnetic field dependence in 3 directions
$$H_B = -B \vec{\sigma} \cdot \vec{n}$$

Now, the Zeeman splitting should create an energy gap at $k_x=0$, such that there are again two (helicity) bands which are split from each other. In the band-gap, the electrons with opposite $k$ can couple like p-waves. However, I'm not too sure about why this happens, I'll need to understand the *Aguado* review to understand the Rashba effect which enable the p-wave coupling.

## 6 May
- Finished Kitaev Chain using Kwant
- Made everything interactive with Holoviews

See <a href='https://nbviewer.jupyter.org/github/arfor/m1internship/blob/master/Jupyter/Kitaev%20Chain.ipynb'>Kitaev Chain.ipynb</a>

## 3 May
- Model Plot of Kitaev Chain just with Python
- Same model using Kwant

Whenever there is a conservation law (symmetry/invariance: time->energy, translation->momentum, rotation->anular momentum), the hamiltonian can be block diagonalised.
Then one can study the topology of the blocks independently. In sublattice symmetry, the number of levels above and below the fermi level are constant and there are no zero-energy crossings -> topologically invariant.
In particle-hole symmetry (e.g. superconductors), also the number of eigenvalues below and above zero are equal, but there exist zero-energy crossings -> there is some topology, but obviously, the number of eigenvalues below zero is not a good topological number since it doesn't change. The right topological invariant is the signature of the *Pfaffian* of the Hamiltonian, which is -1 for odd number of electrons and +1 for an even number.

> **The symmetry of the system determines the type and existence of a topological invariant**
